In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt

import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout  
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [2]:
train = pd.read_csv('datasets/podzielone/ml/stacjonarne/train_ml.csv', index_col=0)
val1 = pd.read_csv('datasets/podzielone/ml/stacjonarne/valid1_ml.csv', index_col=0)
val2 = pd.read_csv('datasets/podzielone/ml/stacjonarne/valid2_ml.csv', index_col=0)
val3 = pd.read_csv('datasets/podzielone/ml/stacjonarne/valid3_ml.csv', index_col=0)
test = pd.read_csv('datasets/podzielone/ml/stacjonarne/test_ml.csv', index_col=0)




train.index = pd.to_datetime(train.index)
val1.index = pd.to_datetime(val1.index)
val2.index = pd.to_datetime(val2.index)
val3.index = pd.to_datetime(val3.index)
test.index = pd.to_datetime(test.index)


train["day_of_week"] = train.index.dayofweek
train["day_of_year"] = train.index.dayofweek
train["week"] = train.index.week
train["quarter"] = train.index.quarter
train["return_ratio"] = train.y_return_ratio.shift()

val1["day_of_week"] = val1.index.dayofweek
val1["day_of_year"] = val1.index.dayofweek
val1["week"] = val1.index.week
val1["quarter"] = val1.index.quarter
val1["return_ratio"] = val1.y_return_ratio.shift()

val2["day_of_week"] = val2.index.dayofweek
val2["day_of_year"] = val2.index.dayofweek
val2["week"] = val2.index.week
val2["quarter"] = val2.index.quarter
val2["return_ratio"] = val2.y_return_ratio.shift()

val3["day_of_week"] = val3.index.dayofweek
val3["day_of_year"] = val3.index.dayofweek
val3["week"] = val3.index.week
val3["quarter"] = val3.index.quarter
val3["return_ratio"] = val3.y_return_ratio.shift()

test["day_of_week"] = test.index.dayofweek
test["day_of_year"] = test.index.dayofweek
test["week"] = test.index.week
test["quarter"] = test.index.quarter
test["return_ratio"] = test.y_return_ratio.shift()

whole = pd.concat([train, val1, val2, val3, test]).asfreq('B', method='bfill').fillna(method='bfill')
val = pd.concat([val1, val2, val3]).asfreq('B', method='bfill').fillna(method='bfill')

train.columns=train.columns.str.replace('(','')
train.columns=train.columns.str.replace(']','')

val1.columns=val1.columns.str.replace('(','')
val1.columns=val1.columns.str.replace(']','')
val2.columns=val2.columns.str.replace('(','')
val2.columns=val2.columns.str.replace(']','')
val3.columns=val3.columns.str.replace('(','')
val3.columns=val3.columns.str.replace(']','')

val.columns=val.columns.str.replace('(','')
val.columns=val.columns.str.replace(']','')


train1 = train.asfreq('B', method='bfill').fillna(method='bfill')
train2 = pd.concat([train, val1]).asfreq('B', method='bfill').fillna(method='bfill')
train3 = pd.concat([train, val1, val2]).asfreq('B', method='bfill').fillna(method='bfill')

val1 = val1.asfreq('B', method='bfill').fillna(method='bfill')
val2 = val2.asfreq('B', method='bfill').fillna(method='bfill')
val3 = val3.asfreq('B', method='bfill').fillna(method='bfill')
test = test.asfreq('B', method='bfill').fillna(method='bfill')



display(train.head(3))
display(val1.head(3))
display(val2.head(3))
display(val3.head(3))
display(test.head(3))
display(whole.head(3))

,y_return_ratio,return_ratio_shift_1,return_ratio_shift_2,return_ratio_shift_3,return_ratio_shift_4,return_ratio_shift_5,return_ratio_shift_6,return_ratio_shift_7,TensorFlow__diff_1_shift_4,deep learning__diff_1_shift_4,...,ATVI_gaming_rr,TTWO_gaming_rr,AMD_rr,SandP_rr,NasdaqTech_rr,day_of_week,day_of_year,week,quarter,return_ratio
2012-07-02,-0.026773,0.007610,0.032207,0.011120,-0.031538,0.012461,-0.045353,0.015861,0.0,0.100000,...,0.028212,0.015062,0.042357,0.011922,0.019487,0,0,27,3,NaN
2012-07-03,0.026022,0.043807,0.007610,0.032207,0.011120,-0.031538,0.012461,-0.045353,0.0,0.033333,...,0.010553,0.000530,0.015901,0.013208,0.011249,1,1,27,3,-0.026773
2012-07-05,-0.010145,-0.026773,0.043807,0.007610,0.032207,0.011120,-0.031538,0.012461,0.0,0.033333,...,0.021303,0.017479,0.026957,0.004373,0.002684,3,3,27,3,0.026022


,y_return_ratio,return_ratio_shift_1,return_ratio_shift_2,return_ratio_shift_3,return_ratio_shift_4,return_ratio_shift_5,return_ratio_shift_6,return_ratio_shift_7,TensorFlow__diff_1_shift_4,deep learning__diff_1_shift_4,...,ATVI_gaming_rr,TTWO_gaming_rr,AMD_rr,SandP_rr,NasdaqTech_rr,day_of_week,day_of_year,week,quarter,return_ratio
2018-01-02,0.030233,0.001166,-0.001368,0.011113,-0.003165,-0.004624,0.003518,-0.009045,-0.032258,0.096774,...,0.001341,0.005920,-0.012784,-0.002025,-0.005463,1,1,1,1,0.030233
2018-01-03,0.065814,-0.019757,0.001166,-0.001368,0.011113,-0.003165,-0.004624,0.003518,-0.129032,0.387097,...,0.007407,0.017018,0.026379,0.003129,0.007868,2,2,1,1,0.030233
2018-01-04,0.005271,0.030233,-0.019757,0.001166,-0.001368,0.011113,-0.003165,-0.004624,-0.032258,0.096774,...,0.015722,0.019918,0.082243,0.005831,0.013572,3,3,1,1,0.065814


,y_return_ratio,return_ratio_shift_1,return_ratio_shift_2,return_ratio_shift_3,return_ratio_shift_4,return_ratio_shift_5,return_ratio_shift_6,return_ratio_shift_7,TensorFlow__diff_1_shift_4,deep learning__diff_1_shift_4,...,ATVI_gaming_rr,TTWO_gaming_rr,AMD_rr,SandP_rr,NasdaqTech_rr,day_of_week,day_of_year,week,quarter,return_ratio
2018-03-01,-0.040455,-0.002109,0.002643,0.015610,0.002650,-0.030392,0.021490,-0.010791,0.285714,-0.071429,...,0.004256,-0.004988,-0.007612,-0.010339,-0.004857,3,3,9,1,-0.040455
2018-03-02,0.018647,-0.016500,-0.002109,0.002643,0.015610,0.002650,-0.030392,0.021490,0.285714,-0.071429,...,-0.007588,-0.027815,-0.024627,-0.013666,-0.012133,4,4,9,1,-0.040455
2018-03-05,-0.003763,-0.040455,-0.016500,-0.002109,0.002643,0.015610,0.002650,-0.030392,0.857143,-0.214286,...,0.000413,-0.007027,-0.028146,-0.007927,-0.005995,0,0,10,1,0.018647


,y_return_ratio,return_ratio_shift_1,return_ratio_shift_2,return_ratio_shift_3,return_ratio_shift_4,return_ratio_shift_5,return_ratio_shift_6,return_ratio_shift_7,TensorFlow__diff_1_shift_4,deep learning__diff_1_shift_4,...,ATVI_gaming_rr,TTWO_gaming_rr,AMD_rr,SandP_rr,NasdaqTech_rr,day_of_week,day_of_year,week,quarter,return_ratio
2018-05-01,0.009960,0.004929,0.039509,-0.020524,-0.011971,-0.021118,-0.001441,-0.031011,0.066667,0.033333,...,-0.002038,0.007704,-0.016584,-0.002772,-0.008446,1,1,18,2,0.009960
2018-05-02,-0.003654,-0.006318,0.004929,0.039509,-0.020524,-0.011971,-0.021118,-0.001441,0.066667,0.033333,...,0.013161,0.017689,0.000912,-0.006153,-0.001804,2,2,18,2,0.009960
2018-05-03,0.029517,0.009960,-0.006318,0.004929,0.039509,-0.020524,-0.011971,-0.021118,0.066667,0.033333,...,0.017768,0.025826,0.005009,-0.001482,0.004321,3,3,18,2,-0.003654


,y_return_ratio,return_ratio_shift_1,return_ratio_shift_2,return_ratio_shift_3,return_ratio_shift_4,return_ratio_shift_5,return_ratio_shift_6,return_ratio_shift_7,TensorFlow__diff_1_shift_4,deep learning__diff_1_shift_4,...,ATVI_gaming_rr,TTWO_gaming_rr,AMD_rr,SandP_rr,NasdaqTech_rr,day_of_week,day_of_year,week,quarter,return_ratio
2018-07-02,0.022541,0.021806,-0.025910,0.012002,-0.047141,-0.023959,-0.019824,0.008225,-0.400000,-0.100000,...,0.018332,0.022284,0.007958,0.005633,0.010789,0,0,27,3,0.022541
2018-07-03,-0.022292,-0.016441,0.021806,-0.025910,0.012002,-0.047141,-0.023959,-0.019824,-0.133333,-0.033333,...,-0.000786,0.002361,-0.014474,-0.002542,-0.001850,1,1,27,3,0.022541
2018-07-04,0.024869,0.022541,-0.016441,0.021806,-0.025910,0.012002,-0.047141,-0.023959,-0.133333,-0.033333,...,0.006027,0.001725,0.008344,0.002730,0.005586,3,3,27,3,-0.022292


,y_return_ratio,return_ratio_shift_1,return_ratio_shift_2,return_ratio_shift_3,return_ratio_shift_4,return_ratio_shift_5,return_ratio_shift_6,return_ratio_shift_7,TensorFlow__diff_1_shift_4,deep learning__diff_1_shift_4,...,ATVI_gaming_rr,TTWO_gaming_rr,AMD_rr,SandP_rr,NasdaqTech_rr,day_of_week,day_of_year,week,quarter,return_ratio
2012-07-02,-0.026773,0.007610,0.032207,0.011120,-0.031538,0.012461,-0.045353,0.015861,0.0,0.100000,...,0.028212,0.015062,0.042357,0.011922,0.019487,0,0,27,3,-0.026773
2012-07-03,0.026022,0.043807,0.007610,0.032207,0.011120,-0.031538,0.012461,-0.045353,0.0,0.033333,...,0.010553,0.000530,0.015901,0.013208,0.011249,1,1,27,3,-0.026773
2012-07-04,-0.010145,-0.026773,0.043807,0.007610,0.032207,0.011120,-0.031538,0.012461,0.0,0.033333,...,0.021303,0.017479,0.026957,0.004373,0.002684,3,3,27,3,0.026022


In [3]:
S = [(train1, val1), (train2, val2), (train3, val3)]

In [4]:
variables = train.columns.tolist()
variables.remove("y_return_ratio")
features = train[variables].columns
target = 'y_return_ratio'

# Korelacje Spearmana

In [5]:
corr_matrix = whole.drop(columns=['y_return_ratio']).corr("spearman").abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

var1 = [col for col in upper.columns if upper[col].max() < 0.9]
var2 = [col for col in upper.columns if upper[col].max() < 0.7]
var3 = [col for col in upper.columns if upper[col].max() < 0.5]
var4 = [col for col in upper.columns if upper[col].max() < 0.4]
var5 = [col for col in upper.columns if upper[col].max() < 0.3]

print(len(var1))
print(len(var2))
print(len(var3))
print(len(var4))
print(len(var5))

41
36
28
24
17


# ExtraTreesRegression

In [6]:
X_train1 = train1[var1]
y_train1 = train1[target]
X_test1 = val[var1]
y_test1 = val[target]

model = ExtraTreesRegressor()
model.fit(X_train1, y_train1)
feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances

/home/michal/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


,importance
SandP_diff_1,0.057498
AD_diff_1,0.044463
return_ratio_shift_7,0.042777
neural network__diff_1_shift_9,0.040144
ATAN_diff_1,0.038764
ATVI_gaming_diff_1,0.035329
return_ratio_shift_3,0.034404
return_ratio_shift_2,0.030657
OBV_diff_1,0.030538
TTWO_gaming_diff_1,0.028765


# LSTM

In [ ]:
scalerX = MinMaxScaler(feature_range=(0, 1))
scalerY = MinMaxScaler(feature_range=(0, 1))
end = int(len(var1)/10)
for i in range(1, 2):
    trainX = train1[feature_importances.index[:i*10]]
    trainY = train1['y_return_ratio']
    
    testX = val[feature_importances.index[:i*10]]
    testY = val['y_return_ratio']
    
    features_set = scalerX.fit_transform(trainX)
    features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))
    labels = scalerY.fit_transform(np.array(trainY).reshape(-1, 1))
    
    model = Sequential()
    
    model.add(LSTM(units=100, return_sequences=True, input_shape=(features_set.shape[1], 1)))
    model.add(Dropout(0.2))
    
    model.add(LSTM(units=100, return_sequences=True))  
    model.add(Dropout(0.2))

    model.add(LSTM(units=100, return_sequences=True))  
    model.add(Dropout(0.2))

    model.add(LSTM(units=100))  
    model.add(Dropout(0.2))
    
    model.add(Dense(units = 1, activation='relu'))
    
    model.compile(optimizer = 'adam', loss = 'mse', metrics=['accuracy'])
    model.fit(features_set, labels, epochs = 100, batch_size = 10)
    
    test_features_set = scalerX.transform(testX)
    test_features_set = np.reshape(test_features_set, (test_features_set.shape[0], test_features_set.shape[1], 1))
    
    predictions = model.predict(test_features_set)
    predictions = scalerY.inverse_transform(predictions)
    
    plt.figure(figsize=(10,6))
    pd.Series([x[0] for x in predictions], index=y_test1.index).plot(label='pred')
    y_test1.plot(label="real")
    plt.legend()
    plt.show()
#     plt.plot(predictions)

Epoch 1/100
1435/1435 [==============================] - 6s 4ms/step - loss: 0.0054 - acc: 6.9686e-04
Epoch 2/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0037 - acc: 6.9686e-04
Epoch 3/100
1435/1435 [==============================] - 4s 2ms/step - loss: 0.0034 - acc: 6.9686e-04
Epoch 4/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0034 - acc: 6.9686e-04
Epoch 5/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0035 - acc: 6.9686e-04
Epoch 6/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0035 - acc: 6.9686e-04
Epoch 7/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0034 - acc: 6.9686e-04
Epoch 8/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.0034 - acc: 6.9686e-04
Epoch 9/100
1435/1435 [==============================] - 5s 3ms/step - loss: 0.0034 - acc: 6.9686e-04
Epoch 10/100
1435/1435 [==============================] - 4s 3ms/step - loss: 0.00

1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 81/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 82/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 83/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 84/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 85/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 86/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 87/100
1435/1435 [==============================] - 3s 2ms/step - loss: 0.0031 - acc: 6.9686e-04
Epoch 88/100
 740/1435 [==============>...............] - ETA: 1s - loss: 0.0038 - acc: 0.0000e+00

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances

In [ ]:
# X_train1 = train1[var1]
# y_train1 = train1[target]
# X_test1 = test[var1]
# y_test1 = test[target]

# model = ExtraTreesRegressor()
# model.fit(X_train1, y_train1)
# feature_importances = pd.DataFrame(model.feature_importances_, index=X_train1.columns, columns=['importance']).sort_values('importance', ascending=False)
# feature_importances